In [1]:
NODE_COUNT = 8
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/non_iid/fedstellar_DFL_21_07_2024_14_14_33/metrics/participant_"
date_str = "21/07/2024 12:14:33" # minus two hours for timezone

In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/non_iid/fedstellar_DFL_21_07_2024_14_14_33/metrics/participant_0
    step      value                     timestamp                    tag
0  11215  99.000000 2024-07-21 12:31:08.112192000  Resources/CPU_percent
1  11225  99.099998 2024-07-21 12:31:18.144019200  Resources/CPU_percent
2  11235  99.199997 2024-07-21 12:31:28.203168768  Resources/CPU_percent
3  11245  99.300003 2024-07-21 12:31:38.366226944  Resources/CPU_percent
4  11255  99.199997 2024-07-21 12:31:48.410894592  Resources/CPU_percent
2024-07-21 12:34:50 2024-07-21 12:14:47
                 step     value                     timestamp
tag                                                          
Test/Accuracy   12461  0.950759 2024-07-21 12:34:48.881328384
Test/F1Score    12461  0.943593 2024-07-21 12:34:48.881361408
Test/Loss       12461  0.110427 2024-07-21 12:34:48.881289472
Test/Precision  12461  0.951261 2024-07-21 12:34:48.881340672
Test/Recall     12461  0.950759 2024

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,12461,0.950759,2024-07-21 12:34:48.881328384,participant_0
Test/F1Score,12461,0.943593,2024-07-21 12:34:48.881361408,participant_0
Test/Loss,12461,0.110427,2024-07-21 12:34:48.881289472,participant_0
Test/Precision,12461,0.951261,2024-07-21 12:34:48.881340672,participant_0
Test/Recall,12461,0.950759,2024-07-21 12:34:48.881351168,participant_0
Test/Accuracy,12366,0.950759,2024-07-21 12:34:49.269963008,participant_1
Test/F1Score,12366,0.943593,2024-07-21 12:34:49.269995008,participant_1
Test/Loss,12366,0.110427,2024-07-21 12:34:49.269925120,participant_1
Test/Precision,12366,0.951261,2024-07-21 12:34:49.269974272,participant_1


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_21012/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,9836.375,0.9508
Test/F1Score,9836.375,0.9436
Test/Loss,9836.375,0.1104
Test/Precision,9836.375,0.9513
Test/Recall,9836.375,0.9508


In [5]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_21012/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,2409.0,0.0
Test/F1Score,2409.0,0.0
Test/Loss,2409.0,0.0
Test/Precision,2409.0,0.0
Test/Recall,2409.0,0.0


In [6]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_21012/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,9836.375,0.9508
Test/F1Score,9836.375,0.9436
Test/Loss,9836.375,0.1104
Test/Precision,9836.375,0.9513
Test/Recall,9836.375,0.9508
